In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import (StructField,StructType, 
                               DateType, IntegerType, FloatType, StringType, TimestampType)
from pyspark.sql.functions import (col, date_format, udf, sum, count,stddev, 
                                   avg, desc, round, format_number, lit)
from funciones import joiner, aggregater, aggregater_filter

In [2]:
data_schema = [StructField("name", StringType(), True),
               StructField("age", IntegerType(), True),
              ]

In [3]:
spark = SparkSession.builder.appName("people").getOrCreate()

In [4]:
final_struc = StructType(fields = data_schema)

In [5]:
df = spark.read.json('people.json', schema=final_struc)

In [6]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)



In [7]:
df.show()

+-------+---+
|   name|age|
+-------+---+
|Michael| 25|
|   Andy| 30|
| Justin| 19|
+-------+---+



In [8]:
df_standev = df.select(round(stddev("age"),2).alias("age_std"))

In [9]:
df_standev.show()

+-------+
|age_std|
+-------+
|   5.51|
+-------+



In [10]:
data_schema = [StructField("id", StringType(), True),
               StructField("origen", StringType(), True),
               StructField("destino", StringType(), True),
               StructField("kilometros", FloatType(), True),
               StructField("precio_km", FloatType(), True),
              ]

In [11]:
final_struc = StructType(fields = data_schema)

In [12]:
df = spark.read.json('diber/Tarea02/data/persona1.json', schema=final_struc)

In [13]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- origen: string (nullable = true)
 |-- destino: string (nullable = true)
 |-- kilometros: float (nullable = true)
 |-- precio_km: float (nullable = true)



In [14]:
df.show()

+---------+------+-------+----------+---------+
|       id|origen|destino|kilometros|precio_km|
+---------+------+-------+----------+---------+
|120230592| 10101|  10301|       6.2|    450.0|
|120230592| 10301|  10107|      10.9|    400.0|
|120230592| 10107|  10401|      40.0|    550.0|
|120230592| 10702|  10104|      35.8|    550.0|
|120230592| 10104|  10203|      12.6|    480.0|
|120230592| 10202|  10302|      14.5|    380.0|
|120230592| 10301|  10309|      18.2|    400.0|
|120230592| 10103|  10201|       6.3|    390.0|
|120230592| 10201|  10105|      11.6|    395.0|
|120230592| 10105|  10110|       7.0|    480.0|
+---------+------+-------+----------+---------+



In [15]:
df = spark.read.json(
    ['diber/Tarea02/data/persona1.json',
     'diber/Tarea02/data/persona2.json',
     'diber/Tarea02/data/persona3.json',
    ], schema=final_struc)

In [16]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- origen: string (nullable = true)
 |-- destino: string (nullable = true)
 |-- kilometros: float (nullable = true)
 |-- precio_km: float (nullable = true)



In [17]:
df.show(30)

+---------+------+-------+----------+---------+
|       id|origen|destino|kilometros|precio_km|
+---------+------+-------+----------+---------+
|700900134| 70101|  70103|      32.0|    600.0|
|700900134| 70103|  70101|      33.1|    610.0|
|700900134| 70101|  70102|      42.3|    590.0|
|700900134| 70102|  10101|     115.1|    700.0|
|700900134| 10101|  10201|      12.6|    500.0|
|700900134| 10201|  10103|      14.5|    380.0|
|700900134| 10103|  10601|      19.3|    500.0|
|700900134| 10101|  10307|      12.6|    440.0|
|700900134| 10301|  10312|      31.8|    430.0|
|700900134| 10312|  10104|      38.7|    465.0|
|111720927| 60115|  10107|      85.7|    550.0|
|111720927| 10107|  10301|      10.7|    410.0|
|111720927| 10401|  10107|      40.3|    560.0|
|111720927| 10104|  10702|      35.9|    570.0|
|111720927| 10203|  10104|      12.8|    495.0|
|111720927| 10302|  10202|      14.6|    375.0|
|111720927| 10309|  10301|      18.1|    415.0|
|111720927| 10201|  10103|       6.5|   

In [18]:
# DataFrame intermedio de códigos postales de viaje origen
# ------------------------------------------------------------------
dataFrame = df
col_group = "origen"
operacion = count
col_oper = "origen"
alias = "Q_viajes"
sort= desc
df_total_viajes_origen = aggregater(dataFrame, col_group, operacion, col_oper, alias, sort)

df_total_viajes_origen = df_total_viajes_origen.withColumnRenamed("origen", "código")

df_total_viajes_origen = df_total_viajes_origen.withColumn("tipo_viaje", lit('origen')) 

df_total_viajes_origen.show()


+------+--------+----------+
|código|Q_viajes|tipo_viaje|
+------+--------+----------+
| 10201|       3|    origen|
| 10101|       3|    origen|
| 10301|       3|    origen|
| 70101|       2|    origen|
| 10104|       2|    origen|
| 10107|       2|    origen|
| 10103|       2|    origen|
| 10105|       2|    origen|
| 70102|       1|    origen|
| 10312|       1|    origen|
| 10309|       1|    origen|
| 10302|       1|    origen|
| 70103|       1|    origen|
| 10110|       1|    origen|
| 10203|       1|    origen|
| 10401|       1|    origen|
| 10202|       1|    origen|
| 60115|       1|    origen|
| 10702|       1|    origen|
+------+--------+----------+



In [19]:
# DataFrame intermedio de códigos postales de viaje destino
# ------------------------------------------------------------------
dataFrame = df
col_group = "destino"
operacion = count
col_oper = "destino"
alias = "Q_viajes"
sort= desc
df_total_viajes_destino = aggregater(dataFrame, col_group, operacion, col_oper, alias, sort)

df_total_viajes_destino = df_total_viajes_destino.withColumnRenamed("destino", "código")

df_total_viajes_destino = df_total_viajes_destino.withColumn("tipo_viaje", lit('destino')) 

df_total_viajes_destino.show()


+------+--------+----------+
|código|Q_viajes|tipo_viaje|
+------+--------+----------+
| 10107|       3|   destino|
| 10301|       3|   destino|
| 10104|       3|   destino|
| 10201|       3|   destino|
| 10103|       2|   destino|
| 10105|       2|   destino|
| 70102|       1|   destino|
| 70101|       1|   destino|
| 10307|       1|   destino|
| 10601|       1|   destino|
| 10101|       1|   destino|
| 70103|       1|   destino|
| 10202|       1|   destino|
| 10309|       1|   destino|
| 10203|       1|   destino|
| 10401|       1|   destino|
| 10312|       1|   destino|
| 10702|       1|   destino|
| 10110|       1|   destino|
| 10302|       1|   destino|
+------+--------+----------+



In [41]:
# Unión de la lista de viajes origen y destino en un solol data frame
# ------------------------------------------------------------------
df_total_viajes = df_total_viajes_origen.union(df_total_viajes_destino)
df_total_viajes = df_total_viajes.select('código', 'tipo_viaje', 'Q_viajes')
df_total_viajes.show(100)


+------+----------+--------+
|código|tipo_viaje|Q_viajes|
+------+----------+--------+
| 10201|    origen|       3|
| 10301|    origen|       3|
| 10101|    origen|       3|
| 70101|    origen|       2|
| 10103|    origen|       2|
| 10104|    origen|       2|
| 10107|    origen|       2|
| 10105|    origen|       2|
| 70102|    origen|       1|
| 10312|    origen|       1|
| 70103|    origen|       1|
| 10309|    origen|       1|
| 10110|    origen|       1|
| 10203|    origen|       1|
| 60115|    origen|       1|
| 10302|    origen|       1|
| 10401|    origen|       1|
| 10202|    origen|       1|
| 10702|    origen|       1|
| 10104|   destino|       3|
| 10201|   destino|       3|
| 10107|   destino|       3|
| 10301|   destino|       3|
| 10103|   destino|       2|
| 10105|   destino|       2|
| 70102|   destino|       1|
| 70101|   destino|       1|
| 10601|   destino|       1|
| 10101|   destino|       1|
| 10312|   destino|       1|
| 10307|   destino|       1|
| 70103|   des

In [42]:
myList = []
myList.append(("persona_max_km", 700900134))
myList

[('persona_max_km', 700900134)]

In [43]:
data_schema = StructType([StructField("metrica", StringType(), True),
               StructField("valor", StringType(), True),
              ])

In [45]:
df_metricas = spark.createDataFrame(data = myList, schema = data_schema)
print(df_metricas)

DataFrame[metrica: string, valor: string]


In [50]:
print('\n' + "="*80)